In [8]:
from collections import defaultdict

# セグメント木
# op: op(x,y) return x*y みたいな演算(関数)
# e: 単位元を返す関数，op(x,e()) = op(e(),x) = x になるようなe
# n: 要素数
# v: 要素の配列
class SegTree:
    def __init__(self, op, e, n, v=None):
        # 要素数
        self._n = n
        # 二項演算
        self._op = op
        # 単位元を返す関数
        self._e = e
        # セグメント木の深さ(根は0)
        self._log = (n - 1).bit_length()
        # 葉の数
        self._size = 1 << self._log
        # セグメント木(0番目の要素は使わない，1番目の要素が根)
        self._d = [self._e()] * (self._size << 1)
        if v is not None:
            for i in range(self._n):
                self._d[self._size + i] = v[i]
            for i in range(self._size - 1, 0, -1):
                self._d[i] = self._op(self._d[i << 1], self._d[i << 1 | 1])
    # p番目の要素をxに変更(pは0-indexed)
    def set(self, p, x):
        p += self._size
        self._d[p] = x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素にxを加算(pは0-indexed)
    def add(self, p, x):
        p += self._size
        self._d[p] += x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素を返す(pは0-indexed)
    def get(self, p):
        return self._d[p + self._size]
    # min(d[l], d[l+1], ..., d[r-1])とargminを返す(l,rは0-indexedの半開区間)
    def argmin(self, l, r):
        res = self._e()
        minv = 10**18
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                if self._d[l] < minv:
                    minv = self._d[l]
                    res = l
                l += 1
            if r & 1:
                r -= 1
                if self._d[r] < minv:
                    minv = self._d[r]
                    res = r
            l >>= 1
            r >>= 1
        # resから葉に向かってidxを求める
        while res < self._size:
            if self._d[res*2] == minv:
                res *= 2
            else:
                res = res*2+1
        return minv, res-self._size
    
# 拠点に帰る時，前拠点を出発したのが何番目の家かを管理する
# i番目の家にいて，プレゼントをK-1個持ってる時のコストの最小値を管理する
# i番目の家から拠点に帰る時，i番目の要素に(i-1)→iのコストを引く
# (K-1)からiまでの要素のコストの最小値とインデックスを求めて，それにi→拠点→i+1のコストを加えて，i+1番目の要素に入れる，i番目から拠点に帰ったからi番目の要素にインデックスを入れる

def main():
    N,K = map(int,input().split())
    Sx,Sy = map(int,input().split())
    # i番目の家から拠点に帰った時，何番目の家から出発したか
    back_go = defaultdict(lambda: -1)
    # i番目の家にいて，プレゼントをK-1個持ってる時のコストの最小値を乗せるセグ木
    seg = SegTree(min, lambda: 10**18, N)
    XY = []
    for _ in range(N):
        x,y = map(int,input().split())
        XY.append((x,y))
    seg.set(0,((Sx-XY[0][0])**2 + (Sy-XY[0][1])**2)**0.5)
    for i in range(N):
        x,y = XY[i]
        if i == 0:
            if N == 1:
                # 1つしか家がない場合，拠点→家→拠点のコストを出力
                print(((Sx-x)**2 + (Sy-y)**2)**0.5 * 2)
                return
            seg.set(1,seg.get(0)*2+((XY[i+1][0]-Sx)**2 + (XY[i+1][1]-Sy)**2)**0.5)
            back_go[i] = 0
            continue
        # i番目の要素に(i-1)→iのコストを引く
        seg.add(i,-((XY[i][0]-XY[i-1][0])**2 + (XY[i][1]-XY[i-1][1])**2)**0.5)
        # (K-1)からiまでの要素のコストの最小値とインデックスを求める
        min_cost,idx = seg.argmin(max(0,i-K+1),i+1)
        # i番目から拠点に帰ったからi番目の要素にインデックスを入れる
        back_go[i] = idx
        if i == N-1:
            continue
        # それにi→拠点→i+1のコストを加えて，i+1番目の要素に入れる
        seg.set(i+1, min_cost + ((XY[i][0]-Sx)**2 + (XY[i][1]-Sy)**2)**0.5 + ((XY[i+1][0]-Sx)**2 + (XY[i+1][1]-Sy)**2)**0.5)
    ans_list = []
    tmp = N
    while tmp != 0:
        ans_list.append(tmp-1)
        tmp = back_go[tmp-1]
        ans_list.append(tmp)
    ans_list.reverse()
    ans = 0
    for i in range(0,len(ans_list),2):
        ans += ((XY[ans_list[i]][0]-Sx)**2 + (XY[ans_list[i]][1]-Sy)**2)**0.5
        for j in range(ans_list[i],ans_list[i+1]):
            ans += ((XY[j][0]-XY[j+1][0])**2 + (XY[j][1]-XY[j+1][1])**2)**0.5
        ans += ((XY[ans_list[i+1]][0]-Sx)**2 + (XY[ans_list[i+1]][1]-Sy)**2)**0.5
    print(ans)

if __name__ == "__main__":
    main()

2.8284271247461903
